<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/MACHINE%20LEARNING%20Y%20DEEP%20LEARNING%20/3_Fine_tuningSentimentSpanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Afinamiento de Modelos con Transformers -  Sentiment Analysis Español**

---





In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os # reinicia kernel
os._exit(00)

In [1]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric, Features, Value, ClassLabel

Uso de datos TASS 2019

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ruta = "/content/drive/MyDrive/Colab Notebooks/MaestriaCienciaDatosV1/MachineLearningAplicado/Sesion15"
#ruta = "/content/drive/My Drive/Colab Notebooks"

In [ ]:
#train_file = ruta+'/train_sent.csv'
#test_file = ruta+'/test_sent.csv'

In [3]:
test_file = '/content/test_sent.csv'
train_file = '/content/train_sent.csv'

In [4]:
data_files = {"train": train_file, "test": test_file}
sentiment_dataset = load_dataset("csv", data_files=data_files, delimiter=",")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
sentiment_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'polarity'],
        num_rows: 966
    })
    test: Dataset({
        features: ['text', 'polarity'],
        num_rows: 498
    })
})

In [6]:
sentiment_dataset['train'][0]

{'text': 'Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
 'polarity': 'P'}

**Actualización Clases (índices)**

In [7]:
classes = {'N':0, 'P':1, 'NEU':2,'NONE':3}
def compute_class_index(example):
    return {"labels": classes[example['polarity']]}

In [8]:
sentiment_dataset = sentiment_dataset.map(compute_class_index)

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [9]:
sentiment_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'polarity', 'labels'],
        num_rows: 966
    })
    test: Dataset({
        features: ['text', 'polarity', 'labels'],
        num_rows: 498
    })
})

In [10]:
sentiment_dataset['train'][0]

{'text': 'Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
 'polarity': 'P',
 'labels': 1}

**Fine-tuning**

In [11]:
checkpoint = "dccuchile/bert-base-spanish-wwm-uncased"#
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Tokenization dataset

In [12]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [13]:
tokenized_datasets = sentiment_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets["train"][:2]

{'text': ['Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
  'Sabes que no tendrás un buen día cuando lo primero que haces en la mañana es tratar de cambiar un foco y este te cae en la cara'],
 'polarity': ['P', 'N'],
 'labels': [1, 0],
 'input_ids': [[4,
   1320,
   1170,
   3185,
   1009,
   3547,
   11591,
   1019,
   1772,
   8982,
   1041,
   1412,
   1091,
   3193,
   1009,
   1032,
   4331,
   1008,
   2066,
   8352,
   1100,
   5197,
   1100,
   1019,
   1040,
   1094,
   3148,
   1091,
   29297,
   6291,
   1008,
   5],
  [4,
   1718,
   1041,
   1054,
   6210,
   1044,
   1274,
   1707,
   1351,
   1086,
   2722,
   1041,
   2919,
   1035,
   1032,
   2243,
   1028,
   4221,
   1009,
   4152,
   1044,
   20323,
   1040,
   1277,
   1155,
   9008,
   1035,
   1032,
   2618,
   5]],
 'token_type_ids': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,

Eliminar columnas adicionales del dataset y transformar en tensores

In [15]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "polarity"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Definición de métrica a utilizar

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    #metric = load_metric("accuracy")  # #accuracy")#"f1","macro")#, "multilabel")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)#,# average='macro')

Definición de argumentos para entrenamiento

In [17]:
#training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=10)# carpeta donde s

In [18]:
training_args = TrainingArguments(
    output_dir="mi_modelo_esp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10, #
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Definición de modelo de Clasificador

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4) # indicar número de clases y modelo

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definición de Trainer

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Entrenamiento

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.122236,0.554217
2,No log,1.110786,0.554217
3,No log,1.119707,0.612450
4,No log,1.314191,0.550201
5,No log,1.562098,0.552209
6,No log,1.650561,0.564257
7,No log,1.814904,0.586345
8,No log,2.001840,0.574297
9,0.445500,2.076389,0.580321
10,0.445500,2.115457,0.568273


TrainOutput(global_step=610, training_loss=0.3670830144256842, metrics={'train_runtime': 207.9438, 'train_samples_per_second': 46.455, 'train_steps_per_second': 2.933, 'total_flos': 201357573086592.0, 'train_loss': 0.3670830144256842, 'epoch': 10.0})

Evaluación

In [ ]:
trainer.evaluate()

{'eval_loss': 3.265194892883301,
 'eval_f1': 0.4956805069029466,
 'eval_runtime': 5.0527,
 'eval_samples_per_second': 98.56,
 'eval_steps_per_second': 12.468,
 'epoch': 10.0}